# Using Neptune together with Amazon SageMaker training jobs 

This tutorial uses some code (with adaptations) from the [official AWS tutorial](https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/scikit_bring_your_own). We age going to show how to add Neptune logging to a custom training job in SageMaker. For this, we are going to create a Docker container with pre-installed Neptune, and adapt Amazon's code by adding Netune logging to it.

## Docker container

Our container is a simplified version of the container in AWS's tutorial. We are using `python` base image and added `neptune-client` and `neptune-sklearn` as additional dependencies.

In [1]:
!cat Dockerfile

FROM python:3-slim-buster

RUN pip --quiet --no-cache-dir install \
    numpy scipy scikit-learn pandas flask gunicorn \
    neptune-client neptune-sklearn

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

# Set up the program in the image
COPY decision_trees /opt/program
WORKDIR /opt/program


### Training script

The training script can be found in `decision_trees/train`. As compared to the script from AWS's tutorial, we added few lines of our code to it:

```diff
[...]

+ import neptune.new as neptune
+ import neptune.new.integrations.sklearn as npt_utils

[...]

def train():
+    run = neptune.init_run()
     [...]
     
     # Now use scikit-learn's decision tree classifier to train the model.
     clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
     clf = clf.fit(train_X, train_y)

+    run["cls_summary"] = npt_utils.create_classifier_summary(
+       clf, train_X, train_X, train_y, train_y
+    )
     [...]
```

### Build the Docker image and push it to ECR

Next, we need to build the container. The following Bash script assumes that you have [AWS CLI](https://aws.amazon.com/cli/) installed on your machine. It again uses the code from the AWS tutorial. It is lengthy, because it automatically creates an ECR repository for us and pushes the image to it.

In [2]:
!cat build_and_push.sh

#!/bin/bash
set -e

# The name of our algorithm
algorithm_name="neptune-sagemaker-demo"

chmod +x decision_trees/train

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"
echo "Image: ${fullname}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region "${region}" | docker login --username AWS --password-stdin "${fullname}"

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .

In [ ]:
!bash ./build_and_push.sh

## Start the training job

<strong><font color='red'>This part of the notebook is to be run from the Amazon SageMaker Notebook.</font></strong>

### Obtaining the Neptune token from AWS Secrets

If you store Neptune token and project name in AWS Secrets, you can read them using the following code. If you do that, you can use your token and project name in the place of `NEPTUNE_API_TOKEN` and `NEPTUNE_PROJECT` in the next cell. Alternativelly, you can add the code below to the `decision_trees/train` script before building the Docker image and read the secrets from the `secrets` dictionary instead of `os.envir`.

If you want to read the secrets form AWS Secrets, make sure that your SageMaker Notebook as a rola that allows for the access to the secrets, in particular the `secretsmanager:GetSecretValue` permission for the appropriate secret.

In [ ]:
import boto3
from botocore.exceptions import ClientError
import json

secret_name = "AmazonSageMaker-tw-neptune-v2"
region_name = "us-east-1"

# Create a Secrets Manager client
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)

# Decrypts secret using the associated KMS key.
secrets = json.loads(get_secret_value_response["SecretString"])

In the example below, we are going to use the anonymous token `neptune.ANONYMOUS_API_TOKEN`, so we first need to install Neptune client to obtain the token.

In [ ]:
!pip install neptune-client

The code below was taken from the official AWS tutorial. The only difference is that we are passing the `NEPTUNE_API_TOKEN` and `NEPTUNE_PROJECT` as environment variables to the [estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html).

In [ ]:
import os
from sagemaker import get_execution_role
import neptune.new as neptune
import sagemaker as sage

# S3 prefix
s3_prefix = "neptune-sagemaker-demo-data"

role = get_execution_role()

sess = sage.Session()

WORK_DIRECTORY = "data"

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=s3_prefix)

account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "{}.dkr.ecr.{}.amazonaws.com/neptune-sagemaker-demo:latest".format(account, region)

tree = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.m5.large",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
    environment={
        "NEPTUNE_API_TOKEN": neptune.ANONYMOUS_API_TOKEN,
        "NEPTUNE_PROJECT": "common/quickstarts"
    }
)

tree.fit(data_location)